# **Dropout Layer Regression - House Price Prediction**

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import tensorflow
import keras
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MinMaxScaler
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import r2_score

In [2]:
df = pd.read_csv('train.csv')
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,NaN,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [4]:
df.isnull().sum()

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64

In [5]:
df.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [6]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['Id', 'SalePrice']),
                                                    df['SalePrice'],
                                                    test_size=0.2,
                                                    random_state=42)

In [7]:
numerical_columns = X_train.select_dtypes(include=['number']).columns.tolist()
categorical_columns = X_train.select_dtypes(include=['object', 'category']).columns.tolist()

numerical_indices = [X_train.columns.get_loc(col) for col in numerical_columns]
categorical_indices = [X_train.columns.get_loc(col) for col in categorical_columns]

In [8]:
handle_numerical = Pipeline(steps=[
    ('scale', MinMaxScaler()),
    ('impute', KNNImputer(n_neighbors=11))
])

In [9]:
handle_categorical = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('encode', OneHotEncoder(drop='first', handle_unknown='ignore'))
])

In [10]:
preprocessing = ColumnTransformer(transformers=[
    ('numerical', handle_numerical, numerical_indices),
    ('categorical', handle_categorical, categorical_indices)
])

In [11]:
X_train = preprocessing.fit_transform(X_train)
X_test = preprocessing.transform(X_test)

c:\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [14, 29] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [12]:
model = Sequential()

In [13]:
model.add(Dense(X_train.shape[1], activation='relu'))
model.add(Dense(X_train.shape[1], activation='relu'))
model.add(Dense(X_train.shape[1], activation='relu'))
model.add(Dense(X_train.shape[1], activation='relu'))
model.add(Dense(X_train.shape[1], activation='relu'))
model.add(Dense(X_train.shape[1], activation='relu'))
model.add(Dense(X_train.shape[1], activation='relu'))
model.add(Dense(X_train.shape[1], activation='relu'))
model.add(Dense(X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='relu'))

In [14]:
model.compile(loss='mean_squared_error', optimizer='Adam')

In [15]:
history = model.fit(X_train, y_train, epochs=100, batch_size=1, validation_data=(X_test, y_test))

Epoch 1/100
1168/1168 ━━━━━━━━━━━━━━━━━━━━ 26s 10ms/step - loss: 18383826944.0000 - val_loss: 8677086208.0000
Epoch 2/100
1168/1168 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - loss: 11658764288.0000 - val_loss: 7033704448.0000
Epoch 3/100
1168/1168 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - loss: 10598748160.0000 - val_loss: 10285716480.0000
Epoch 4/100
1168/1168 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - loss: 9943149568.0000 - val_loss: 6754293248.0000
Epoch 5/100
1168/1168 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - loss: 7741528576.0000 - val_loss: 4218912768.0000
Epoch 6/100
1168/1168 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - loss: 6433271808.0000 - val_loss: 5806375424.0000
Epoch 7/100
1168/1168 ━━━━━━━━━━━━━━━━━━━━ 20s 9ms/step - loss: 7131403776.0000 - val_loss: 1996015616.0000
Epoch 8/100
1168/1168 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - loss: 4699278336.0000 - val_loss: 4964448256.0000
Epoch 9/100
1168/1168 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - loss: 5612787712.0000 - val_loss: 4502349824.0000
Epoch 10/100
1168/1168

In [16]:
y_pred = model.predict(X_test)

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step


In [17]:
r2_score(y_test, y_pred)

0.8005534410476685